# Zadanie:


## Zistiť z dát, ktoré platby predstavujú mzdy a nájsť ďalšie hlavné kategórie pre platby 

In [1]:

# Importovanie knižníc

import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



In [2]:

# Načítanie dát a vytvorenie názvov pre jednotlivé stĺpce

data = pd.read_table('data_for_applicants.txt', header = None, 
                     names = ['date', 'client', 'amount', 'sender'])


In [3]:

# pohľad na dáta

data.head()


,date,client,amount,sender
0,2014-09-12,651959088,6686,627986531
1,2014-09-17,725866593,14462,222382928
2,2014-09-12,313567829,9965,222382928
3,2014-09-12,855015364,10835,222382928
4,2014-09-03,414606140,18320,222382928


* **date** --  dátum

* **client**  -- id klienta

* **amount**  -- výška platby

* **sender**  -- id odosielateľa

## 1. Krok 

* Stanovenie minimálnej mzdy. V roku 2014 bola minimálna mzda v ČR 8500 korún

* Vyselektovanie platieb vyšších ako min. mzda do premennej **high_income**

In [4]:

min_mzda = 8500

# podmnožina high_income obsahuje všetky platby vyššie ako minimálna mzda + ostatné stĺpce

high_income = data[ data['amount'] > min_mzda ]


## 2. Krok

* Konvertovanie dátumov z formátu rok-mesiac-deň na rok-mesiac. Stĺpec **date** bude obsahovať iba 6 kategórii (mesiacov), čo uľahčí hľadanie výplat (výplatu hodnotím ako pravidelnú platbu, ktora bola klientovi pripísaná na účet každý mesiac aspoň od jedného odosielateľa) <br/> <br/>

* Pridanie nového stĺpca **join_id** do podmnožiny **high_income**, ktorý bude obsahovať spárované id klienta a id odosieteľa. Vytvorím tak jedninečné id páry medzi klientom a odosielateľom, ktoré mi pomôžu pri selekcii výplat <br/> <br/> 

* Následné vytvorenie nových premenných (**months, id**) s jedinečnými hodnotami pre mesiace a id páry. Obidve premenné vložím do **for loop** cyklu. Cieľ cyklu je získanie iba takých id párov, ktoré sa nachádzajú v každom mesiaci. Použijem na to funkciu intersection. Výsledné id páry mi pomôžu vyselektovať len takých klientov, ktorí dostávajú platby vyššie ako min. mzda každý mesiac 

In [5]:

# formátovanie dátumov

date = pd.to_datetime( high_income['date'] )
high_income['date'] = date.dt.strftime('%Y-%m')

# vytvorenie nového stĺpca ('join_id'), ktorý bude obsahovať id páry medzi klientom a odosielateľom

high_income['join_id'] = high_income.client.map(str) + high_income.sender.map(str)

# vytvorene premenných, ktoré obsahujú unikátne dátumy a spojené id páry

months = set(high_income['date'])
id = set(high_income['join_id'])

# cyklus, ktorého cieľom je nájsť všetky id páry nachádzajúce sa v každom mesiaci

for m in months:
    month = high_income[ high_income['date'] == m ]
    id = id.intersection(month['join_id']) 
    
    
# pohľad na id páry

print(list(id)[0:5])


['814910129838639938', '821357916674455418', '132302670356081381', '893860660559111653', '592384784308204383']


## 3. Krok

* Použitie id párov, ktoré sa nachádzajú v premennej **id** na získanie novej podmnožiny **stable_income**. Nová podmnožina bude obsahovať všetkých klientov, ktorí dostávajú každý mesiac platby minimálne od jedného odosielateľov. Každý odosielateľ platí klientovi každý mesiac minimálne jednu čiastku, ktorá prevyšuje minimálnu mzdu <br/> <br/>

* Následne sa vytvorí nová premenná **client_senders**, ktorá obsahuje počty jedinečných odosielateľov pre kažďeho klienta. Táto premenná sa použije na vyselektovanie klientov, ktorí majú len jedného odosielateľa (**one_sender**) alebo viacerých odosielateľov (**multi_sender**)

In [6]:

# vyselektovanie dát z high_income podľa id párov z premennej 'id', ktorá pochádza z for loop cyklu

stable_income = high_income[ high_income['join_id'].isin(id) ]

# vytvorenie premennej, ktorá obsahuje počty unikátnych odosielateľov pre každého klienta

client_senders = stable_income.groupby('client')['sender'].nunique()

# premenná, ktorá obsahuje id klientov len s jedným odosielateľom  

one_sender = client_senders[ client_senders == 1 ].index

# v premennej multi_sender sa nachádzajú klienti, ktorí dostávajú platby od viacerých odosielateľov každý  mesiac

multi_sender = client_senders[ client_senders > 1 ].index


### Tabuľka početnosti klientov s unikátnymi odosielateľmi 

In [7]:

# v tabuľke na ľavej strane sú skupiny vyjadrujúce množstvo jedinečných odosielateľov pre klientov 

# v pravej časti je celkový počet klientov, ktorí majú dané množstvo unikátnych odosielateľov prislúchajúce na ľavej strane

# napr. 3. riadok z výstupu vyjadruje celkové množstvo klientov, ktorí majú troch unikátnych odosielateľov

client_senders.value_counts()

1     15033
2      1808
3       103
4         2
11        1
Name: sender, dtype: int64

## 4. Krok 

* Vytvorenie dvoch nových podmnožín zo **stable_income** na základe id klientov z premenných **one_sender** a **multi_sender** <br/> <br/>

* Dostanem podmnožinu **income_one_sender**, ktorá obsahuje klientov len s jedným odosielateľom platiacim každý mesiac klientovi čiastku prevyšujúcu 8500 korún. <br/> <br/>

* Druhá podmnožina **income_multi_sender** bude obsahovať klientov s viacerými odosielateľmi, ktorí posielajú klientovi každý mesiac peniaze v hodnote prevyšujúcej minimálnu mzdu 


In [8]:

# prvá podmnožina obsahuje klientov len z jedným odosielateľom platiacim každý mesiac

income_one_sender = stable_income[ stable_income['client'].isin(one_sender) ]

# druhá podmnožina obsahuje klientov, ktorí dostávajú platby od viacerých odosielateľov každý mesiac

income_multi_sender = stable_income[ stable_income['client'].isin(multi_sender) ]


### Pohľad na klientov s jedným alebo s viacerými odosielateľmi, ktorí posielajú klientom každý mesiac čiastku vyššiu ako min. mzda

In [9]:

# prvá podmnožina je zoradená podľa id klientov a dátumov, obsahuje klienov s jedným odosielateľom

income_one_sender.sort(['client', 'date']).head(12)


,date,client,amount,sender,join_id
105145,2014-04,56847,11432,222382928,56847222382928
102734,2014-05,56847,11437,222382928,56847222382928
99788,2014-06,56847,11434,222382928,56847222382928
96986,2014-07,56847,11434,222382928,56847222382928
94791,2014-08,56847,11339,222382928,56847222382928
91341,2014-09,56847,11371,222382928,56847222382928
69484,2014-04,61441,17032,547166056,61441547166056
68814,2014-05,61441,45773,547166056,61441547166056
65786,2014-06,61441,17821,547166056,61441547166056
62836,2014-07,61441,17445,547166056,61441547166056


In [10]:

# druhá podmnožina obsahuje klientov s viacerými odosielateľmi

income_multi_sender.sort(['client', 'date']).head(12)


,date,client,amount,sender,join_id
70642,2014-04,450431,12704,219047249,450431219047249
327953,2014-04,450431,14748,633236581,450431633236581
327954,2014-04,450431,16117,633236581,450431633236581
68106,2014-05,450431,12704,219047249,450431219047249
326953,2014-05,450431,15095,633236581,450431633236581
326954,2014-05,450431,16359,633236581,450431633236581
65081,2014-06,450431,12706,219047249,450431219047249
326077,2014-06,450431,12842,633236581,450431633236581
326078,2014-06,450431,16303,633236581,450431633236581
62153,2014-07,450431,12779,219047249,450431219047249


## 5. Krok

* Vytvorenie zvyšnej podmnožiny (**unstable_income**) kde sa nachádzajú klienti, ktorým chýba odosielateľ platiaci každý mesiac čiastku prevyšujúcu minimálnu mzdu. V danej podmnožine klienti môžu prijímať každý mesiac sumu vyššiu ako je minimálna mzda, ale musia ju prijímať minimálne od dvoch odosielateľov. <br/> <br/>

* Podmnožinu **unstable_income** dostanem z **high_income** kde sa nachádzajú všetky platby vyššie ako minimálna mzda. Budem na to potrebovať spoločné indexy z podmnožín **income_one_sender** a **income_multi_sender**, ktoré odčítam od indexov z **high_income**. Dostanem klientov, ktorí nemajú pravidelné mesačné platby od jedného alebo viacerých odosielateľov <br/> <br/>


In [30]:

# spoločne indexy z income_ine_sender a income_multi_sender pridelené do premennej income_all

income_all = pd.concat([income_one_sender, income_multi_sender], axis=0)

# podmnožina unstable_income bude obsahovať indexy, ktoré sa nachádazjú v high_income, ale nenachádzajú sa v income_all
# dostanem klientov s odosielateľmi, ktorí im neposialajú každý  mesiac čiastku prevyšujúcu min. mzdu

unstable_income = high_income[~ high_income.index.isin(income_all.index)]


### Pohľad na dáta v unstable_income podmnožine

In [12]:
unstable_income.sort(['client', 'date']).head(14)

,date,client,amount,sender,join_id
285509,2014-04,61441,15346,45437871,6144145437871
243911,2014-05,61441,9703,762490320,61441762490320
243104,2014-06,61441,8875,762490320,61441762490320
240922,2014-08,61441,11008,762490320,61441762490320
397989,2014-04,72755,9981,566495527,72755566495527
54286,2014-04,78274,68655,482592832,78274482592832
104502,2014-04,87884,21397,222382928,87884222382928
48037,2014-06,106137,10301,627986531,106137627986531
45180,2014-07,106137,11864,627986531,106137627986531
40155,2014-08,106137,25278,345882789,106137345882789


## 6. Krok

* Vytvorenie premennej **client_month_incomes** z **unstable_incomes**, ktorá bude obsahovať klientov a počet mesiacov, v ktorých klienti dostali platbu prevyšujúcu min. mzdu. Ak sa bude id klienta nachádzať vo všetkých šiestich mesiacoch, tak daný klient bude príjimať každý mesiac platbu vyššiu ako je minimálna mzda. Jeho odosielatelia budú minimálne dvaja a ich platby nebude klient dostávať každý mesiac. Napr. jeden odosielateľ bude klientovi posielať peniaze iba 4 mesiace a zvyšné 2 mesiace bude posielať druhý odosielateľ <br/> <br/>

* Pohľad na počet klientov, ktorí su zaradení do skupín mesiacov, v ktorých dostali platbu prevyšujúcu min. mzdu. 

In [13]:

# premenná, ktorá obsahuje id klientov a ich výskyt v danom počte mesiacov kedy dostali platbu  

client_month_incomes = unstable_income.groupby('client')['date'].nunique()

# pohľad na počet klientov, ktorí dostali platby len v určitom počte mesiacov

client_month_incomes.value_counts()


1    7765
6    3016
2    2575
5    2049
4    1696
3    1549
Name: date, dtype: int64

V tabuľke hore, napr. v 4. riadku je 2049 klientov, ktorí dostali platbu v piatich mesiacoch, ale v jednom mesiaci platbu nedostali

## 7. Krok

* Selekcia id klientov z **client_month_incomes**, ktorí dostali každý mesiac platbu prevyšujúcu min. mzdu do premennej **client_id** <br/> <br/>

* ID klientov použijem na vytvorenie novej podmnožiny **six_month_clients** z *unstable_income*. To bude zároveň posledná podmnožina s klientami, ktorí dostali každý mesiac príjem vyšší ako minimálna mzda. Ostatní klienti nedostali každý mesiac čiastku, ktorá by prevyšovala hodnotu minimálnu mzdy. Vytvorím pre nich podmnožinu **other_incomes**

In [14]:

# id klientov, ktorí každý mesiac prijali platbu vyššiu ako min. mzda

client_id = client_month_incomes[ client_month_incomes == 6].index

# vytvorenie podmnoziny six_month_clients cez spoločných id klientov medzi unstable_income a client_id 

six_month_clients = unstable_income[unstable_income['client'].isin(client_id)]

# zvyšná skupina klientov, ktorí nedostali platby vyššie ako min. mzda každy mesiac 

other_incomes = unstable_income[~ unstable_income['client'].isin(client_id)]


### Pohľad na podmnožinu six_month_clients

In [15]:
six_month_clients.sort(['client', 'date']).head(12)

,date,client,amount,sender,join_id
168804,2014-04,1177887,14853,784440758,1177887784440758
167892,2014-05,1177887,13774,136454232,1177887136454232
167115,2014-06,1177887,14985,136454232,1177887136454232
166230,2014-07,1177887,10649,136454232,1177887136454232
165594,2014-08,1177887,13903,136454232,1177887136454232
164540,2014-09,1177887,14394,136454232,1177887136454232
421948,2014-04,2088495,12377,773266812,2088495773266812
421605,2014-05,2088495,11401,773266812,2088495773266812
218683,2014-06,2088495,13985,536955217,2088495536955217
217753,2014-07,2088495,24421,536955217,2088495536955217


## Zahrnutie klientov do segmentov <br/>

* a) Prvý segment obsahuje klientov, ktorí prijali každý mesiac platby prevyšujúce min. mzdu len od jedného odosielateľa. Tvorí ho **income_one_sender** <br/> <br/>

* b) Druhý segment obsahuje klientov s viacerými odosielateľmi, ktorí im posielali platby každý  mesiac. Ak klient prijímal peniaze od dvoch odosielateľov, tak obidvaja odosielatelia posielali klientovi každý mesiac na účet čiastku prevyšujúcu min. mzdu. Tento segment tvorí **income_multi_sender**<br/> <br/>

* c) Tretí segment obsahuje klientov, ktorí síce dostali každý mesiac príjem vyšši ako min. mzda, ale ich odosielatelia im neposielali peniaze každý mesiac. Klienti v tomto segment prijímali platby najmenej od dvoch odosielateľov. Tretí segment tvorí **six_month_clients**  <br/> <br/>

* d) Posledný segment obsahuje klientov, ktorým chýbal minimálne jeden príjem za určitý mesiac prevyšujúci min. mzdu. V ostatných mesiacoch mohli prijať čiastky prevyšujúce min. mzdu. Prehľad aké množstvo klientov prijalo platby v danom počte mesiacov je znázornení v tabuľka v 6. kroku. Posledný segment tvorí **other_incomes** <br/> <br/>

* Klientov z posledných dvoch segmentov využijem aj na zaradenie do nižších platových segmentov 

## Ďalší segment - dôchodky, soc. dávky, brigády, part-time a nízke príjmy

V ďalších segmentoch sa budú nachádzať klienti s nižšími pravidelnými príjmami ako je min. mzda. Klienti nebudú patriť do prvých dvoch segmentov **a)** a **d)** spomenutých vyššie, ale môzu patriť do segmentov **c)** a **d)**. Platby v nových segmentoch môžu byť dôchodky, sociálne dávky, platby z brigád a podobne. Pri selekcii platieb stanovím minimálnu hranicu 3100 korún, čo predstavovalo v roku 2014 neoficiálne minimálny dôchodok v ČR. V ďalšom kroku nájdem id klientov, ktorí prijímali každý mesiac platby vyššie ako min. dôchodok a zároveň nižšie ako min. mzda. Postupujem podobne ako v 2. kroku. Platby označujem ako dôchodky len pre zjednodušenie.

In [31]:

# prvé dva segmenty spomenuté vyššie boli spojené do premennej income_all v 5. kroku. Z tejto premennej použijem 
# id klientov na selekciu nových klientov, ktorí nepatria do income_all, ale nachádzajú sa v pôvodných dátach

client_selection = data[~ data['client'].isin(income_all.client)]

# premenná pension obsahuje platby vyššie ako min. dôchodok, ale nižšie ako min. mzda

min_penzia = 3100
pension = client_selection[ (client_selection['amount'] > min_penzia) & (client_selection['amount'] <= min_mzda) ] 

# formátujem dátumi na rok-mesiac 

date = pd.to_datetime( pension['date'] )
pension['date'] = date.dt.strftime('%Y-%m')

# premmené, ktoré vložím do for loop cyklu

months = set(pension['date'])
id = set(pension['client'])

# for loop cyklus na selekciu id klientov, ktorí prijímali "penziu" každý  mesiac

for m in months:
    month = pension[ pension['date'] == m ]
    id = id.intersection(month['client']) 

# vytvorenie segmentu s klientami, ktorí každý mesiac dostali "penziu" od 1 a viac odsielateľov

stable_pension_income = pension[ pension['client'].isin(id) ]


### Pohľad na dáta klientov s pravidelnými mesačnými "penziami" 

In [17]:
stable_pension_income.sort(['client', 'date']).head(18)

,date,client,amount,sender
52493,2014-04,3306,5404,222382928
48841,2014-05,3306,5404,222382928
321072,2014-05,3306,4216,550277640
45872,2014-06,3306,5348,222382928
320127,2014-06,3306,5503,550277640
43008,2014-07,3306,5404,222382928
319208,2014-07,3306,5591,550277640
40843,2014-08,3306,5403,222382928
318568,2014-08,3306,6957,550277640
37508,2014-09,3306,5400,222382928


Ostatných klientov, ktorí neprijímali "penziu" pravidelne každý mesiac prenesiem do segmentu z označením **unstable_pension**. Budú to klienti, ktorí sa nenechádzajú v **stable_pension_income**, ale sú  prítomní v **pension** 

In [18]:
unstable_pension = pension[~ pension['client'].isin(stable_pension_income['client'])]

### Pohľad na klientov s nepravidelnými mesačnými platbami vo výške 3100-8500 korún


In [19]:
unstable_pension.sort(['client', 'date']).head(8)

,date,client,amount,sender
53350,2014-04,106137,6326,345882789
51034,2014-05,106137,6226,345882789
48036,2014-06,106137,6321,345882789
87643,2014-04,503312,4164,31873657
85758,2014-05,516929,4181,439913156
408544,2014-09,516929,4460,813580157
405448,2014-07,842767,5193,635276731
118480,2014-06,958936,5160,963808981


V poslednom segmente sú klienti, ktorých platby nepresahujú výšku minimálneho dôchodku. Vyselektujem ich z **high_incomes** a **pension** kde sa nachádzajú klienti, ktorí dostali aspoň jednu platbu vyššiu ako min. dôchodok. Posledný segment označím ako **other_low_incomes**

In [26]:

# spojenie id klientov z high_income (z 1. kroku) a z pension

higher_incomes = pd.concat([high_income.client, pension.client], axis=0)

# posledný segment s klientami, ktorí nedostali ani jednu platbu vyššiu ako min. dôchodok

other_low_incomes = data[~ data['client'].isin(higher_incomes)]


### Pohľad na posledný segment

In [27]:
other_low_incomes.sort(['client', 'date']).head(10)


,date,client,amount,sender
66340,2014-06-11,120424,195,544975132
63308,2014-07-11,120424,194,544975132
58071,2014-08-11,120424,201,544975132
57436,2014-09-11,120424,200,544975132
357754,2014-06-12,295940,281,233423418
229886,2014-04-13,484522,414,849713090
229491,2014-05-11,484522,428,849713090
228780,2014-06-17,484522,431,849713090
228010,2014-07-10,484522,419,849713090
226733,2014-08-17,484522,420,849713090


## Zhrnutie segmentov <br/> 

* Podľa výšky platieb: | **0-3100** | **3100-8500** | **8500+** | <br/> <br/>



Segmenty **a), b), c), d)** už boli vysvetlené vyššie a patria do kategórie **8500+** <br/> <br/>

Ďalšie segmenty pre kategóriu **3100-8500** <br/> 

e) Segment **stable_pension_income** obsahuje klientov, ktorí každý mesiac dostali platbu ("penziu") vo výške 3100-8500  korún od jedného alebo viacerých odosielateľov. V tomto segmente môžu byť zahrnutý klienti zo segmentov **c)** a **d)**

f) Segment **unstable_pension** obsahuje klientov, ktorí nedostali "penziu" každý mesiac <br/> <br/>


Posledný  segment pre kategóriu **0-3100** <br/> 

g) Segment **other_low_incomes** obsahuje klientov, ktorí majú všetky platby nižšie ako min. dôchodok. 

